In [1]:
import MeCab
import CaboCha
import pandas as pd
from tqdm import tqdm 
import regex
import re

In [2]:
def get_word(tree, chunk):
    surface = ''
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞':
            surface += token.surface
        elif features[0] == '形容詞':
            surface += features[6]
            break
        elif features[0] == '動詞':
            surface += features[6]
            break
    return surface

def get_2_words(line):
    cp = CaboCha.Parser('-f1')
    tree = cp.parse(line)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1

    tuples = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface =  get_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface = get_word(tree, to_chunk)
            tuples.append((from_surface, to_surface))
    return tuples

In [3]:
#口コミを１文ずつに切り分けし、その文章内を係り受け解析し（単語A,単語B）にkawaii_wordがあればリストに加える
def taples_kawaii(txt):
    kawaii_word = ['可愛い','かわいい','カワイイ','美人','綺麗','きれい','美しい','美女']
    sen = regex.split(r'(?<=[。？])(?!$)', txt, flags=regex.VERSION1)
    #sen = re.split(r'(?<=[。？])(?!$)', txt)
    add_list=[]
    for i in sen:
        tuples = get_2_words(i)
        for j in tuples:
            for k in kawaii_word:
                if k in j:
                    add_list.append(j)
    return add_list

In [4]:
#形態素解析するための関数を定義
tagger = MeCab.Tagger('-Owakati')#タグはMeCab.Tagger（neologd辞書）を使用したい
tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [5]:
#kawaii_wordが含まれた（単語A,単語B）をさらに形態素解析し、tennin_wordが含まれていたらフラグ立てをする
def tennin_hantei(i):

    txt = tokenize(i, 2, 10000, True)

    vector_list =[]

    a = 0

    tennin_word = ['店員','女性店員','スタッフ','女将','女性','店主','助手','奥さん','奥様','店長']
    for k in txt:
        if k in tennin_word:
            a+=1
        else:
            a+=0        
    if a >=1:
        kawaii_flg = 1
    else:
        kawaii_flg = 0

    return kawaii_flg

In [6]:
df = pd.read_csv('all_results.csv')
df['store_score'].astype(float)
df = df[df['store_score'] > 4.7]
df = df.dropna(subset=['dinner_review'])
df = df.drop(['detail', 'lunch_review', 'score'], axis=1)
df.head()

,dinner_review,store_name,store_score
7716,\n 六本木の【鮨 さいとう】さんに伺いました。地下鉄六本木一丁目駅の3番...,鮨 さいとう,4.72
7717,\n この日は久々の鮨さいとうへ(*･ω･)ﾉ今回はちょっと特別な会。普段...,鮨 さいとう,4.72
7718,\n 友人が予約を取ってくれたところに飛び入りで参加してきました。こちらの...,鮨 さいとう,4.72
7723,\n 六本木の【鮨さいとう】さんに伺いました。今回も齋藤さんのお席でつまみ...,鮨 さいとう,4.72
7724,\n 六本木一丁目の【鮨 さいとう】さんに伺いました。いつも通りつまみから...,鮨 さいとう,4.72


In [7]:
pd.set_option("display.max_colwidth", 400)
list_sentence = df['dinner_review'].tolist()

new_list=[]
for txt in tqdm(list_sentence):
    new_list.append(taples_kawaii(txt))
kawaii_flg_list2=[]
kawaii_flg_list3=[]
for k in new_list:
    kawaii_flg_list=[]
    if len(k)==0:
        kawaii_flg_list.append(0)
    else: 
        for i in k:
                kawaii_flg_list.append(tennin_hantei(i))
                if tennin_hantei(i)==1:
                    kawaii_flg_list3.append(sum(kawaii_flg_list))

df['kawaii_flg']=kawaii_flg_list3
df_ramen = df.groupby(['store_name','store_score'])['dinner_review'].apply(list).apply(' '.join).reset_index().sort_values('store_score', ascending=False)
df_ramen2 = df.groupby(['store_name','store_score'])['kawaii_flg'].sum().reset_index().sort_values('kawaii_flg', ascending=False)
df_ramen2

100%|██████████████████████████████████████████████████████████████████████████████| 1078/1078 [05:59<00:00,  3.00it/s]


ValueError: Length of values does not match length of index

In [55]:
kawaii_flg_list

[0, 0]

In [56]:
new_list

[[],
 [],
 [],
 [('美味しい', '美しい'), ('美しい', '兼ね備える')],
 [('切りつける', '美しい'), ('ファルム全て', '美しい'), ('美しい', '美味しい')],
 [],
 [],
 [('ホント', '美しい'), ('美しい', '流線型')],
 [],
 [('美しい', 'フォルム')],
 [],
 [],
 [('感動する', '美しい'), ('所作', '美しい')],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('早い', '美しい'), ('美しい', '美味しい')],
 [],
 [],
 [('おいしい', '美しい')],
 [],
 [],
 [('蒸す', '可愛い'), ('酸味', '可愛い'), ('器', '可愛い'), ('可愛い', '＼(^_^)／')],
 [('', '美しい'), ('握り', '美しい'), ('美しい', '新しい')],
 [],
 [('美しい', '納得レベル')],
 [],
 [],
 [],
 [],
 [],
 [],
 [('綺麗', 'カウンター')],
 [('所作', '美しい'), ('美しい', '増す')],
 [('申し分ない', '美しい'), ('フォーム', '美しい'), ('大将譲り', '美しい'), ('', '美しい')],
 [],
 [('トロ', '綺麗'), ('綺麗', '甘み'), ('非常', '綺麗'), ('綺麗', 'ハーモニー')],
 [],
 [('綺麗', '店構え'), ('', '綺麗'), ('綺麗', '全て')],
 [],
 [('美しい', '良い')],
 [],
 [('綺麗', '形'), ('美しい', 'フォーム')],
 [],
 [],
 [],
 [],
 [('美しい', '女性'), ('美しい', 'ない')],
 [('鮪', '美しい'), ('美しい', '見惚れる')],
 [],
 [('店', '綺麗'), ('綺麗', '癒す')],
 [],
 [],
 [],
 [('所作', '美しい'), ('美しい', '見惚れる')],
 [],
 [('きれい', '美味しい')],
